In [1]:
import os

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from rich import print
from pytorch_tabular.utils import load_covertype_dataset, print_metrics

# %load_ext autoreload
# %autoreload 2

# Cover Type Dataset

Predicting forest cover type from cartographic variables only (no remotely sensed data). The actual forest cover type for a given observation (30 x 30 meter cell) was determined from US Forest Service (USFS) Region 2 Resource Information System (RIS) data. Independent variables were derived from data originally obtained from US Geological Survey (USGS) and USFS data. Data is in raw form (not scaled) and contains binary (0 or 1) columns of data for qualitative independent variables (wilderness areas and soil types).

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so that existing forest cover types are more a result of ecological processes rather than forest management practices.

It is from [UCI ML Repository](https://archive.ics.uci.edu/ml/datasets/covertype), but with small changes:
- The one hot encoded columns are converted to categorical - Soli Type and Wilderness type

In [2]:
data, cat_col_names, num_col_names, target_col = load_covertype_dataset()
train, test = train_test_split(data, random_state=42)
train, val = train_test_split(train, random_state=42)

In [3]:
print(f"Data Shape: {data.shape} | # of cat cols: {len(cat_col_names)} | # of num cols: {len(num_col_names)}")
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")
print(f"[bold dodger_blue2] Features: {num_col_names + cat_col_names}[/bold dodger_blue2]")
print(f"[bold purple4]Target: {target_col}[/bold purple4]")

Data Shape: (581012, 13) | # of cat cols: 2 | # of num cols: 10

Train Shape: (326819, 13) | Val Shape: (108940, 13) | Test Shape: (145253, 13)

 Features: ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 
'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 
'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type']

Target: Cover_Type

# Importing the Library

In [4]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import GANDALFConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)
from pytorch_tabular.models.common.heads import LinearHeadConfig

## Define the Configs

This is the most crucial step in the process. There are four configs that you need to provide(most of them have intelligent default values), which will drive the rest of the process.

* DataConfig - Define the target column names, categorical and numerical column names, any transformation you need to do, etc.
* ModelConfig - There is a specific config for each of the models. This determines which model we are going to train and also lets you define the hyperparameters of the model
* TrainerConfig - This let's you configure the training process by setting things like batch_size, epochs, early stopping, etc. The vast majority of parameters are directly borrowed from PyTorch Lightning and is passed to the underlying Trainer object during training
* OptimizerConfig - This let's you define and use different Optimizers and LearningRate Schedulers. Standard PyTorch Optimizers and Learning RateSchedulers are supported. For custom optimizers, you can use the parameter in the fit method to overwrite this. The custom optimizer should be PyTorch compatible
* ExperimentConfig - This is an optional parameter. If set, this defines the Experiment Tracking. Right now, only two experiment tracking frameworks are supported: Tensorboard and Weights&Biases. W&B experiment tracker has more features like tracking the gradients and logits across epochs.


<div class="alert alert-block alert-warning"> <b>NOTE</b> Categorical and Continuous columns defined are the features and should not contain the target columns. </div>

In [5]:
data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    accelerator="auto",  # can be 'cpu','gpu', 'tpu', or 'ipu'
)
optimizer_config = OptimizerConfig()

# Best practice: Use head config to define the head. This will help you not pass
# unnecessary params to the model
head_config = LinearHeadConfig(
    layers="",  # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming",
).__dict__  # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

model_config = GANDALFConfig(
    gflu_stages=6,
    gflu_feature_init_sparsity=0.3,
    gflu_dropout=0.0,
    task="classification",
    head="LinearHead",  # Linear Head
    head_config=head_config,  # Linear Head Config
    learning_rate=1e-3,
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-12-31 14:33:09,990 - {pytorch_tabular.tabular_model:132} - INFO - Experiment Tracking is turned off

## Training the Model 
Now that we have defined the configs and the TabularModel. We just need to call the `fit` method and pass the train and test dataframes. We can also pass in validation dataframe. But if omitted, TabularModel will separate 20%(also configurable) at random from the data as validation.

By default, EarlyStopping is enabled and is monitoring Validation Loss with a patience of 3 epochs. The trainer also saves the best model(based on validation loss) and loads that model at the end of training. `TrainerConfig` has the parameters to tweak this default behaviour.

In [6]:
tabular_model.fit(train=train, validation=val)

Seed set to 42


2023-12-31 14:33:10,064 - {pytorch_tabular.tabular_model:504} - INFO - Preparing the DataLoaders

2023-12-31 14:33:10,232 - {pytorch_tabular.tabular_datamodule:498} - INFO - Setting up the datamodule for          
classification task

2023-12-31 14:33:10,617 - {pytorch_tabular.tabular_model:554} - INFO - Preparing the Model: GANDALFModel

2023-12-31 14:33:10,682 - {pytorch_tabular.tabular_model:320} - INFO - Preparing the Trainer

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ GANDALFBackbone  │ 42.4 K │
│ 1 │ _embedding_layer │ Embedding1dLayer │    896 │
│ 2 │ _head            │ Sequential       │    252 │
│ 3 │ loss             │ CrossEntropyLoss │      0 │
└───┴──────────────────┴──────────────────┴────────┘

Trainable params: 43.6 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 43.6 K                                                                                               
Total estimated model params size (MB): 0

Output()

2023-12-31 14:37:33,091 - {pytorch_tabular.tabular_model:643} - INFO - Training the model completed

2023-12-31 14:37:33,092 - {pytorch_tabular.tabular_model:1456} - INFO - Loading the best model

## Evaluating the Model

### Loss and Metrics on New Data
To evaluate the model on new data on the same metrics/loss that was used during training, we can use the `evaluate` method

In [7]:
result = tabular_model.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

/home/manujosephv/miniconda3/envs/lightning_upgrade/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9138330817222595     │
│         test_loss         │    0.21494710445404053    │
└───────────────────────────┴───────────────────────────┘

## New Predictions as DataFrame
To get the prediction as a dataframe, we can use the `predict` method. This will add predictions to the same dataframe that was passed in. For classification problems, we get both the probabilities and the final prediction taking 0.5 as the threshold

In [8]:
pred_df = tabular_model.predict(test)
pred_df.head()

,1_probability,2_probability,3_probability,4_probability,5_probability,6_probability,7_probability,prediction
250728,0.999410,0.000333,1.296315e-09,3.981011e-10,2.254133e-08,1.912545e-07,2.569415e-04,1
246788,0.056541,0.928676,6.810129e-05,2.655586e-09,1.468230e-02,3.267031e-05,1.109751e-08,2
407714,0.000614,0.955515,3.976083e-04,2.862627e-07,2.562141e-02,1.785192e-02,1.472094e-10,2
25713,0.080934,0.919064,9.255725e-10,1.176771e-10,2.239666e-08,6.145486e-07,7.067687e-07,2
21820,0.000007,0.730117,2.615734e-04,7.821862e-04,2.688144e-01,1.747545e-05,5.805644e-09,2


In [9]:
print_metrics(
    metrics=[
        (accuracy_score, "Accuracy", {}),
        (f1_score, "F1", {"average": "macro"}),
    ],
    y_true=test[target_col],
    y_pred=pred_df["prediction"],
    tag="Holdout",
)

Holdout Accuracy: 0.9138262204567204 | Holdout F1: 0.8608195764007365


## Saving and Loading the Model

In [10]:
tabular_model.save_model("examples/basic")

2023-12-31 14:43:50,661 - {pytorch_tabular.tabular_model:1500} - WARNING - Directory is not empty. Overwriting the 
contents.

In [11]:
loaded_model = TabularModel.load_model("examples/basic")

2023-12-31 14:43:50,992 - {pytorch_tabular.tabular_model:157} - INFO - Experiment Tracking is turned off

2023-12-31 14:43:51,002 - {pytorch_tabular.tabular_model:320} - INFO - Preparing the Trainer

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
result = loaded_model.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

/home/manujosephv/miniconda3/envs/lightning_upgrade/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9138330817222595     │
│         test_loss         │    0.21494710445404053    │
└───────────────────────────┴───────────────────────────┘